In [0]:
from pyspark.sql import SparkSession # Puerta de entrada para trabajar con spark <-- SIEMPRE DEBEMOS IMPORTAR LA LLAVE MAESTRA QUE INICIA TODO.
from pyspark.sql.functions import *  # Funciones propias del módulo SQL de Spark, para trabajar sobre Dataframes.
from pyspark.sql.types import *
spark = SparkSession.builder.appName("OperacionesDataframesApacheSparkDatabricks").getOrCreate() 
"""
^          ^__________^        ^_________^                               ^
|                |                   |                                   | 
Variable   Constructor de Sesión   Nombre Aplicación       Evita conflicto del SparkSession"""


### DATAFRAMES (Versión Databricks Free Edition)

#### Link de [Databrick Free Edition](https://dbc-89f542f8-2df6.cloud.databricks.com/?o=758509963140561)

##### OPERACIONES BÁSICAS CON DATAFRAMES EN DATABRICKS

In [0]:
# Trabajaremos con este dataframe
diccionario = {
  "Nombre":["Pepe","Juan","Pedro","Carlos","Raul"],
  "Edad":[20,25,26,24,21],
  "Nota":[10,12,15,14,10]
}
# En caso los tipos de datos de cada columna sean variados,
# los definimos aparte y luego lo integramos en createDataFrame(schema=**)
from pyspark.sql.types import * ## Importamos esta librería para la creación de estructuras desde cero.
schema_dataframe = StructType([ #⬅️ Método para definir nuevos tipos de estructuras, en este caso, columnas de un DF.
  StructField("Nombre",StringType(),True), #⬅️ Método para definir columna: StructField(NombreColumna,TipoDato,True)
  StructField("Edad",LongType(),True),
  StructField("Nota",LongType(),True)
])
df_example = spark.createDataFrame(data=list(zip(*diccionario.values())),schema=schema_dataframe) 
df_example.show()

In [0]:
##============== Obtener valor o valores de una columna en un Dataframe
df_example[["Edad"]].show() # Obtenemos el valor de una columna
df_example[["Nombre","Edad"]].show() # Obtenemos valores de varias columnas

In [0]:
##============== Fitrar valores en una o varias columnas: método ➡️.filter(col(NombreColumna)&|Condición)
# Filtramos los registros con la edad mayor a 21...
df_example.filter(col("Edad")>21).show() # Mediante la función col(NombreColumna), accedemos a la columna y sus filas. 
# Filtrar valores en dos columnas específicas
df_example.filter(
  (col("Edad")==25) & (col("Nombre")=='Juan')
).show() # La condición es separada en tuplas y con un operador lógico.

In [0]:
##============== Agregar nuevas columnas al Dataframe: método ➡️.withColumn(NombreColumna,Condición/Valor)
# Utilizaremos lit para valores literales y/o asignar valores
df_example = df_example.withColumn( #⬅️ Agregamos una sola columna
  "Estado", # Nuevo nombre de la columna
  lit("Activo") # Agregar el mismo valor para la N cantidad de filas
) 
df_example.show()

#⬅️ Agregamos varias columnas: método ➡️.withColumns({NombreColumna:Condición/Valor})
df_example = df_example.withColumns({ 
  "Estado": # Nuevo nombre de la columna 1
  lit("Activo"), # Agregar el mismo valor para la N cantidad de filas
  "Fecha Registro": # Nuevo nombre de la columna 2
  lit("2025-07-15"), # Agregar el mismo valor para la N cantidad de filas
}) 
df_example.show()

##=============== Agregar nueva columna al Dataframe en base a una condición: método ➡️.withColumn(NombreColumna,Condición)
df_example = df_example.withColumn(
  "Categorid_Edad",                                        # Nombre de nueva columna
  when(col("Edad")<18,lit("Menor"))                        # Primera condición
  .when((col("Edad")>=18) & (col("Edad")<25),lit("Joven")) # Segunda condición
  .otherwise(lit("Adulto"))                                # Última condición
)
df_example.show()

In [0]:
##=============== Seleccionamos una o más columnas del dataframe: método ➡️.select(col(NombreColumna))
df_example = df_example.select(
  col("Nombre"),                   # Seleccionamos columna Nombre
  col("Edad"),                     # Seleccionamos columna Edad
)
df_example.show()

In [0]:
##=============== Renombrar una columna de un DataFrame: 
# ➡️método .withColumnRenamed(existing=NombreColumnaActual,new=NombreNuevoColumna) ||
# ➡️método .withColumnsRenamed({NombreAntiguoExistente1:NombreNuevoColumna1,NombreAntiguoExistente2:NombreNuevoColumna2}) 

df_example = df_example.withColumnRenamed(existing="Nombre",new="Name") #⬅️ Renombra columnas individuales
df_example = df_example.withColumnsRenamed({"Nombre":"Name","Edad":"Age"}) #⬅️ Renombra varias columnas
# df_example = df_example.selectExpr("Nombre AS Name","Edad as Age") 
#⬅️ .selectExpr() renombra la o las columnas, sin embargo, selecciona esas renombradas y omite el resto (No recomendado) 
df_example.show()

In [0]:

##=============== Eliminar columnas de un Dataframe: ➡️ método .drop(col(NombreColumna)) 
df_example = df_example.drop(col("Nota")) # Ingresar nombre de columna a eliminar
df_example.show()

In [0]:
##=============== Ordenar Dataframe por una columna en específico: ➡️ método .sort(col(NombreColumna).asc() || .desc())
df_example.orderBy(col("Edad").desc()).show() # Forma descendente por la columna Edad
df_example.orderBy(col("Edad").asc()).show() # Forma ascendente por la columna Edad
